In [10]:
import pandas as pd
import requests
from datetime import datetime
import time
import random

Storing our base URL, Token, and our header for API requests

In [ ]:
TOKEN = token  # type: ignore
BASE_URL = 'https://www.ncei.noaa.gov/cdo-web/api/v2/'
HEADERS = {
    'token': TOKEN
}

# token is not here to keep token secure

Now we want to map the FIPS  codes (codes for each county) to the county name

In [24]:
CA_COUNTY_MAPPING = {
    'FIPS:06001': 'Alameda',
    'FIPS:06003': 'Alpine',
    'FIPS:06005': 'Amador',
    'FIPS:06007': 'Butte',
    'FIPS:06009': 'Calaveras',
    'FIPS:06011': 'Colusa',
    'FIPS:06013': 'Contra Costa',
    'FIPS:06015': 'Del Norte',
    'FIPS:06017': 'El Dorado',
    'FIPS:06019': 'Fresno',
    'FIPS:06021': 'Glenn',
    'FIPS:06023': 'Humboldt',
    'FIPS:06025': 'Imperial',
    'FIPS:06027': 'Inyo',
    'FIPS:06029': 'Kern',
    'FIPS:06031': 'Kings',
    'FIPS:06033': 'Lake',
    'FIPS:06035': 'Lassen',
    'FIPS:06037': 'Los Angeles',
    'FIPS:06039': 'Madera',
    'FIPS:06041': 'Marin',
    'FIPS:06043': 'Mariposa',
    'FIPS:06045': 'Mendocino',
    'FIPS:06047': 'Merced',
    'FIPS:06049': 'Modoc',
    'FIPS:06051': 'Mono',
    'FIPS:06053': 'Monterey',
    'FIPS:06055': 'Napa',
    'FIPS:06057': 'Nevada',
    'FIPS:06059': 'Orange',
    'FIPS:06061': 'Placer',
    'FIPS:06063': 'Plumas',
    'FIPS:06065': 'Riverside',
    'FIPS:06067': 'Sacramento',
    'FIPS:06069': 'San Benito',
    'FIPS:06071': 'San Bernardino',
    'FIPS:06073': 'San Diego',
    'FIPS:06075': 'San Francisco',
    'FIPS:06077': 'San Joaquin',
    'FIPS:06079': 'San Luis Obispo',
    'FIPS:06081': 'San Mateo',
    'FIPS:06083': 'Santa Barbara',
    'FIPS:06085': 'Santa Clara',
    'FIPS:06087': 'Santa Cruz',
    'FIPS:06089': 'Shasta',
    'FIPS:06091': 'Sierra',
    'FIPS:06093': 'Siskiyou',
    'FIPS:06095': 'Solano',
    'FIPS:06097': 'Sonoma',
    'FIPS:06099': 'Stanislaus',
    'FIPS:06101': 'Sutter',
    'FIPS:06103': 'Tehama',
    'FIPS:06105': 'Trinity',
    'FIPS:06107': 'Tulare',
    'FIPS:06109': 'Tuolumne',
    'FIPS:06111': 'Ventura',
    'FIPS:06113': 'Yolo',
    'FIPS:06115': 'Yuba'
}

CA_COUNTIES = list(CA_COUNTY_MAPPING.keys())
# The counties codes will be seperately stored in list format for ease of using

In [ ]:
def fetch_county_data(county_fips, county_name, start_date, end_date):
        """
        Fetch county data TAVG, TMAX, TMIN, PRCP
        """
        url = f'{BASE_URL}data'
        all_results = []
        offset = 1  

        # Goal is to use pagination, we want to keep requesting data till there's nothing left
        # we start at offset 1 which is first 1000, then if more data increase to 1001
        # we repeat this process till 1. Error, or 2. Break then move to next county

        while True: #fetch pages till out of data
            parameters = {
                'datasetid': 'GSOM',
                'locationid': county_fips,
                'startdate': start_date,
                'enddate': end_date,
                'datatypeid': 'TAVG,TMAX,TMIN,PRCP',
                'limit': 1000, # 1000 per request
                'units': 'metric',
                'offset': offset # 1, 1001, 2001..
            }
            
            success = False 
            data = None

            for attempt in range(5): #retry mechanism because of rate limit
                try: 
                    time.sleep(1)
                    # make request
                    response = requests.get(url, headers=HEADERS, params=parameters, timeout=60)
                    response.raise_for_status()
                    data = response.json()
                    success = True # successful 

                    if 'results' in data: # process the data
                        all_results.extend(data['results'])
                        print(f"{len(data['results'])} records : on offset {offset}")
                        total = data.get('metadata', {}).get('resultset', {}).get('count', 0)
                        if offset + 1000 > total: 
                            return all_results 
                        offset += 1000 
                        time.sleep(1)
                        break # exit the loop and move to next page
                    else: 
                        return all_results
                except requests.exceptions.HTTPError as e: 
                    if e.response.status_code == 503: # if encounter 503 error (server side error)
                        wait_time = 10 * (attempt + 1)  # for each attempt increment by 10
                        print(f"Encountered 503 error waiting {wait_time}") # increase wait time and try again
                        time.sleep(wait_time)
                    else: 
                        print(f"HTTP Error: {e}") # if not 503 error just leave it, means bug in code
                        return all_results 
                except Exception as e: 
                    print(f"Error : {e}")
                    if attempt < 4: # if not last attempt
                        time.sleep(5)
                        # continue retry
                    else: 
                        return all_results # give up if last attempt
            if not success: # if not successful then break and move on
                break

        return all_results


In [ ]:

def fetch_california_weather_data(start_year=2020, end_year=2025):
    """
    Fetch weather data for all 58 california counties, combine into dataframe
    """
    start_date = f'{start_year}-01-01' # can adjust if we want other years
    end_date = f'{end_year}-12-31'
    all_data = [] # store data for ALL 58 counties
    failed_counties = [] # track counties that had problem
    for i, (county_fips, county_name) in enumerate(CA_COUNTY_MAPPING.items(), 1 ): # loop through all counties
        print(f"{i}/{len(CA_COUNTY_MAPPING)} : {county_name} county") # to track what county we are on

        county_data = fetch_county_data(county_fips,county_name, start_date, end_date) # fetch county data for county we are on
        if county_data: # process the county data if we got any
            # add county info to each record
            for record in county_data:
                record['county_fips'] = county_fips
                record['county_name'] = county_name
            all_data.extend(county_data) # add to main list
            print(f"Collected {len(county_data)} records \n")
        else: 
            failed_counties.append((county_fips, county_name)) # failed counties
            print(f"No data retreived \n")
        
        if i < len(CA_COUNTY_MAPPING): # wait two seconds before moving to next county
            time.sleep(2)
        
    if not all_data: # check if got data
        print("No data")
        return pd.DataFrame()
    
    # ----- convert to dataframe -----
    df =pd.DataFrame(all_data) # convert list of dictionaries into pd df 
    df['date'] = pd.to_datetime(df['date']) # parse dates

    # change TAVG, TMAX, TMIN, PRCP that are a part of datatype olumn make into own column
    df_pivot = df.pivot_table(
        index = ['county_name', 'county_fips', 'date', 'station'],
        columns = 'datatype',
        values = 'value',
        aggfunc = 'mean' # if station has duplicate entries for same date average
    ).reset_index()
       
    df_pivot.columns.name = None # clean datatype label

    # reordering columns county_name | county_fips | date | station | TAVG | TMAX | TMIN | PRCP
    weather_cols = ['TAVG', 'TMAX', 'TMIN', 'PRCP'] 
    existing_weather = [col for col in weather_cols if col in df_pivot.columns]
    final_cols = ['county_name', 'county_fips', 'date', 'station'] + existing_weather
    df_pivot = df_pivot[[col for col in final_cols if col in df_pivot.columns]]
        
    df_pivot = df_pivot.sort_values(['county_name', 'date']).reset_index(drop=True)    # sort by date and county name alphabetically

    print(f"{df_pivot['county_name'].nunique()}/58") # print summary
    if failed_counties:
        print(f"{len(failed_counties)}")
        for fips, name in failed_counties:
            print(f"  - {name}")
    
    return df_pivot


In [29]:
df_weather = fetch_california_weather_data(start_year=2020, end_year=2025)


1/58 : Alameda county
1000 records : on offset 1
1000 records : on offset 1001
1000 records : on offset 2001
476 records : on offset 3001
Collected 3476 records 

2/58 : Alpine county
1000 records : on offset 1
1000 records : on offset 1001
757 records : on offset 2001
Collected 2757 records 

3/58 : Amador county
872 records : on offset 1
Collected 872 records 

4/58 : Butte county
1000 records : on offset 1
987 records : on offset 1001
Collected 1987 records 

5/58 : Calaveras county
Encountered 503 error waiting 10
447 records : on offset 1
Collected 447 records 

6/58 : Colusa county
Encountered 503 error waiting 10
440 records : on offset 1
Collected 440 records 

7/58 : Contra Costa county
1000 records : on offset 1
1000 records : on offset 1001
630 records : on offset 2001
Collected 2630 records 

8/58 : Del Norte county
1000 records : on offset 1
63 records : on offset 1001
Collected 1063 records 

9/58 : El Dorado county
1000 records : on offset 1
1000 records : on offset 1001

In [30]:
df_weather

,county_name,county_fips,date,station,TAVG,TMAX,TMIN,PRCP
0,Alameda,FIPS:06001,2020-01-01,GHCND:US1CAAL0001,NaN,NaN,NaN,75.1
1,Alameda,FIPS:06001,2020-01-01,GHCND:US1CAAL0003,NaN,NaN,NaN,56.8
2,Alameda,FIPS:06001,2020-01-01,GHCND:US1CAAL0012,NaN,NaN,NaN,29.3
3,Alameda,FIPS:06001,2020-01-01,GHCND:US1CAAL0018,NaN,NaN,NaN,65.9
4,Alameda,FIPS:06001,2020-01-01,GHCND:US1CAAL0025,NaN,NaN,NaN,54.0
...,...,...,...,...,...,...,...,...
73521,Yuba,FIPS:06115,2025-10-01,GHCND:USW00093205,17.5,24.1,11.0,33.8
73522,Yuba,FIPS:06115,2025-11-01,GHCND:US1CAYB0001,NaN,NaN,NaN,227.5
73523,Yuba,FIPS:06115,2025-11-01,GHCND:USC00048606,10.3,15.6,4.9,225.0
73524,Yuba,FIPS:06115,2025-11-01,GHCND:USR0000CPIK,12.7,16.7,8.6,NaN


In [ ]:
def create_monthly_summary(df_weather): 
    df = df_weather.copy() # create copy to not modify original df
    
    #extract year and month
    df['year'] = df['date'].dt.year 
    df['month'] = df['date'].dt.month

    # define colmns to average
    weather_cols = ['TAVG', 'TMAX', 'TMIN', 'PRCP']
    cols_to_avg = [col for col in weather_cols if col in df.columns]

    # group records by county, year, month then average
    df_monthly = df.groupby(['county_name', 'county_fips', 'year', 'month'])[cols_to_avg].mean().reset_index()
    df_monthly = df_monthly.sort_values(['county_name', 'year', 'month']).reset_index(drop=True)

    return df_monthly



In [33]:
df_monthly_summary = create_monthly_summary(df_weather)
df_monthly_summary

,county_name,county_fips,year,month,TAVG,TMAX,TMIN,PRCP
0,Alameda,FIPS:06001,2020,1,9.700000,13.941667,5.466667,49.389474
1,Alameda,FIPS:06001,2020,2,12.466667,18.250000,6.658333,0.000000
2,Alameda,FIPS:06001,2020,3,11.725000,16.116667,7.358333,50.083333
3,Alameda,FIPS:06001,2020,4,14.533333,19.491667,9.558333,35.242105
4,Alameda,FIPS:06001,2020,5,18.133333,24.158333,12.108333,18.200000
...,...,...,...,...,...,...,...,...
4108,Yuba,FIPS:06115,2025,7,23.233333,30.800000,15.600000,0.250000
4109,Yuba,FIPS:06115,2025,8,24.966667,32.933333,17.033333,2.040000
4110,Yuba,FIPS:06115,2025,9,21.833333,28.666667,14.966667,5.325000
4111,Yuba,FIPS:06115,2025,10,14.733333,20.366667,9.133333,61.275000


In [ ]:
def create_seasonal_summary(df_monthly_summary):

    df = df_monthly_summary.copy() # copy to not modify orignal df
    
    # define the season months
    season_map = {
        12: 'Winter', 1: 'Winter', 2: 'Winter',
        3: 'Spring', 4: 'Spring', 5: 'Spring',
        6: 'Summer', 7: 'Summer', 8: 'Summer',
        9: 'Fall', 10: 'Fall', 11: 'Fall'
    }

    # map the months to the seasons
    df['season'] = df['month'].map(season_map)
    
    # define columns to average
    weather_cols = ['TAVG', 'TMAX', 'TMIN', 'PRCP']
    cols_to_avg = [col for col in weather_cols if col in df.columns]
    
    # group by county, year, seasons then average
    df_seasonal = df.groupby(['county_name', 'county_fips', 'season'])[cols_to_avg].mean().reset_index()
    
    # order by winter, spring, summer, fall
    season_order = ['Winter', 'Spring', 'Summer', 'Fall']
    df_seasonal['season'] = pd.Categorical(df_seasonal['season'], categories=season_order, ordered=True)

    # sort the results
    df_seasonal = df_seasonal.sort_values(['county_name', 'season']).reset_index(drop=True)
    
    # put columns in order county_name, county_fips , season , TAVG, TMAX , TMIN, PRCP
    final_cols = ['county_name', 'county_fips', 'season'] + cols_to_avg
    df_seasonal = df_seasonal[final_cols]
    
    return df_seasonal
    

In [ ]:
df_seasonal_summary = create_seasonal_summary(df_monthly_summary)
df_seasonal_summary.to_csv('seasonal_summary_final.csv', index=False)
# now named seasonal_summary_temp_celsius

In [ ]:
df_seasonal = pd.read_csv('seasonal_summary_final.csv')

# Celsius to Fahrenheit
# F = (C × 9/5) + 32
temp_columns = ['TAVG', 'TMAX', 'TMIN']

for col in temp_columns:
    if col in df_seasonal.columns:
        df_seasonal[col] = (df_seasonal[col] * 9/5) + 32

# the updated data
df_seasonal.to_csv('seasonal_summary_final_fahrenheit.csv', index=False)
# now named seasonal_summary_temp_fahrenh